1. Kütüphanelerin ve Datanın Yüklenmesi

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('../input/airbnb-recruiting-new-user-bookings/train_users_2.csv.zip')
test = pd.read_csv('../input/airbnb-recruiting-new-user-bookings/test_users.csv.zip')
#sessions=pd.read_csv('../input/airbnb-recruiting-new-user-bookings/sessions.csv.zip')
#countries=pd.read_csv('../input/airbnb-recruiting-new-user-bookings/countries.csv.zip')
#age_gender=pd.read_csv('../input/airbnb-recruiting-new-user-bookings/age_gender_bkts.csv.zip')

In [ ]:
Id = 'id' # Submission'da kullanmak üzere id kısmını değişkene atayarak datamdan sileceğm.

mysubmission_ID = test.loc[:,Id]

train.drop(Id, axis=1, inplace=True)
test.drop(Id, axis=1, inplace=True)

# Test ve Train datasını işaretleyebilmek için Train kolonu oluşturdum.
train.loc[:,'Train'] = 1
test.loc[:,'Train'] = 0

# Tek bir df üzerinden gitmek için birleştiriyorum.

df = pd.concat([train,test], ignore_index=True)


2. Datayı Anlama

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
df.info() # Boş değeri olan sütunları belirlemek ve data tiplerini anlamak için info'ya bakıyorum. 

Date_first_booking =NaN olanlar Country_destination'da "NDF" olan kişiler ile aynı. Bu da airbnb'ye kayıt olup, henüz herhangi bir yere gitmeyenleri gösteriyor. Test datasında da bu değerler boş; amaç test datasındaki kişilerin gidecekleri ilk yer bulmak olduğu için bu da beklenebilecek bir durum. Dolayısı ile aslında modelde "Date_first_booking" kolonunu kullanmak anlamlı değil. Yapılabilecek bir şey, sadece bir yere gidenler üzerinden model kurmak. Herhangi başka bir şeyi değiştirmeden, bir de "NDF" olanları çıkarmadan aynı modeli çalıştırarak sonucun nasıl değiştiği gözlemlenebilir. Ayrıca data_first_booking kullanılabilecek bir feature haline getirilerek de model kurulabilir. Örn; uygulamada aktif olduğu tarih ile ilk booking'ini yaptığı tarih arasında benzerlik olan kişiler benzer davranışlar gösteriyor mu? sorusunun cevabı aranabilir.

date_first_booking haricinde, age ve first_affiliate_tracked kolonlarında eksik data görülüyor. Diğer sütunların unique değerlerine baktığımızda, gender'da da "unknown" girilen, eksik veri olarak değerlendirilebileceğimiz data olduğunu görebiliriz.

In [ ]:
train.info() 

In [ ]:
test.info()

In [ ]:
train.describe()

 Yaş datasında hatalı girişler olduğunu gördüm. Max=2014 ve min=1 idi. Yaş datasını gidilen ülke ve cinsiyet ile ilişkilendirebilir miyim diye baktım. Fark yaratan bir şey yoktu. Airbnb'in kurallarına göre 18 yaşından küçük kişiler airbnb aracılığı ile bir yerde konaklayamaz. Bu nedenle 18 yaşından küçük olup, bir yere gitmeyen kişileri datadan kaldırdım. Bir yere giden kişilerin yaşını ise 18'e getirdim.

4 basamaklı girilen yaşların yanısıra 100-116 arasında yaşlar da vardı. Train datasında 105 yaşında bariz bir fazlalık olduğunu gördüm. Eğer boş değer olmasaydı, yaş girmeyenleri sistem otomatik 2015-1910 gibi sabit bir değerle doldurmuş diye düşünecektim. Yaş değerli bir bilgi olacağı için outlier yaratabilecek bu değerleri kaldırdım. Bir sonraki aşamada bu yaş civarında olanların ortak bir özelliği var mı diye bakmayı planlıyorum.

In [ ]:
plt.figure(figsize=(50,30))
sns.countplot(x='age', data=df)

1000 - 1996 arasında olan yaş değerlerinde kişilerin doğum tarihlerini yaş olarak girdiğini kabul ettim. 2014 yılına ait daha fazla değer olduğu için 2014 yılında 18 yaşında olmayı seyahat şartı gibi aldım.18 yaşından küçük olup bir yere gitmemiş olan 136 kişiyi datadan çıkardım.

In [ ]:
dft=df[df['Train']==1]
a=dft[(dft['age']<=17) & (dft['country_destination']=='NDF')].sort_values(by=['age']).index

In [ ]:
df.drop(a,inplace=True)

In [ ]:
df.loc[df['age']<=17, 'age']=18 #test datasında da en küçük yaş =18 oluyor bu kod ile

1000 - 1996 arasında olan yaş değerlerinde kişilerin doğum tarihlerini yaş olarak girdiğini kabul ettim. 2014 yılına ait daha fazla değer olduğu için 2014 yılında 18 yaşında olmayı seyahat şartı gibi aldım.

In [ ]:
def ageconverter(age):
    if (age<1997) & (age>1000):
        return (2014-age)
    else:
        return age

In [ ]:
df.describe()

In [ ]:
df['aged'] =df['age'].apply(ageconverter)
df.drop('age', inplace=True, axis=1)

In [ ]:
df.drop(df[(df['country_destination']=='NDF') & (df['aged']>=1000)].index, inplace=True) #2001-2014 yaş arasındakilerden NDF olanları datadan sildim, 
                                                                                            #diğerlerini nan ile doldurdum.

In [ ]:
df.loc[df['aged']>=1000, 'aged']=np.nan

In [ ]:
df['aged'].describe() #hala yaşı mükün olmayacak byüklükte olan kişiler vardı

In [ ]:
plt.figure(figsize=(30,15))
sns.countplot(x='aged', data=df)

In [ ]:
df.loc[df['aged']>=85, 'aged']=np.nan #grafiğe bakınca 85'ten sonrasını almamaya karar verdim, onları da NaN ile doldurdum.

Yaşta hala NaN değerlerim vardı,cinsiyet ile ilişkisi var mı diye baktım. Sadece train datasına bakmak için dft diye bir df tanımladım. Buna göre

In [ ]:
dft=df[df['Train']==1]

In [ ]:
sns.boxplot(x='gender', y='aged', data =df[df['Train']==1]) # Yaş 'missing' değerlerini ne ile doldurabilirim diye çeşitli featurelara baktım

In [ ]:
dft.groupby('affiliate_channel')['aged'].mean()

En fark edilir değişiklik "affiliate_channel"a göre idi, ona göre doldurdum.

In [ ]:
def agefiller(param):
    age = param[0]
    ac = param[1]
    if pd.isnull(age)==True:
        if ac == 'api':
            return 33
        elif ac== 'content':
            return 41
        elif ac== 'direct':
            return 36
        elif ac=='other':
            return 37
        elif ac== 'remarketing':
            return 40
        elif ac== 'sem-brand':
            return 39
        elif ac== 'sem-non-brand':
            return 40
        elif ac== 'seo':
            return 35
    else:
        return param[0]
    
    
        

In [ ]:
df['aged'] =df[['aged', 'affiliate_channel']].apply(agefiller, axis=1)

In [ ]:
df['aged']=df['aged'].astype(int) # daha okunur olması için integer'a çevirdim

Bir diğer missing data da "'first_affiliate_tracked" özelliğinde idi.

In [ ]:
dft=df[df['Train']==1]
dft.pivot_table(values='gender',index='affiliate_channel', columns='first_affiliate_tracked',aggfunc='count')

affiliate_channel'a göre bakınca, NaN değerleri doldururken seo ise linked, omg ise sem-non-brand, diğerlerini de 'untracked' olarak doldurmaya kadar verdim.

In [ ]:
def fatfiller(param):
    fat= param[0]
    ac = param[1]
    if pd.isnull(fat)==True:
        if ac == 'seo':
            return 'linked'
        elif ac== 'sem-non-brand':
            return 'omg'
        else:
            return 'untracked'
    else:
        return param[0]

In [ ]:
df['first_affiliate_tracked'] =df[['first_affiliate_tracked', 'affiliate_channel']].apply(fatfiller, axis=1)

Bir de boş görünmese de içindeki verilere bakınca first_browser'da da unknown değerler olduğunu gördüm.

In [ ]:
df['first_browser'].replace('-unknown-',np.nan,inplace=True)

In [ ]:
dft.pivot_table(values='language',index='first_browser', columns='first_device_type',aggfunc='count')

first_browser'ın first_device_type ile ilişkisine baktığımda ayırtedici farklar vardı, datamı buna göre dolduracağım bir fonksiyon yazdım.

In [ ]:
def fbfiller(param):
    fb= param[0]
    fdt = param[1]
    if pd.isnull(fb)==True:
        if fdt == 'Android Phone':
            return 'Android Browser'
        elif fdt =='Android Tablet':
            return 'IE'
        elif fdt =='SmartPhone (Other)':
            return 'BlackBerry Browser'
        elif fdt =='iPad':
            return 'Mobile Safari'
        elif fdt =='iPhone':
            return 'Mobile Safari'
        elif fdt =='Mac Desktop':
            return 'Safari'
        else:
            return 'Chrome'
    else:
        return param[0]

In [ ]:
df['first_browser'] =df[['first_browser', 'first_device_type']].apply(fbfiller, axis=1) # fonksiyona göre datamı doldurdum.

3. EDA 

Kişilerin ilk aktif olduğu tarihlere göre akışa bakmak istedim.

In [ ]:
from datetime import datetime

In [ ]:
first_active_date =[]
for i in df['timestamp_first_active']:
    d = datetime.strptime(str(i),'%Y%m%d%H%M%S')
    day_string = d.strftime('%Y-%m-%d')
    first_active_date.append(day_string)

In [ ]:
first_active_time =[]
for i in df['timestamp_first_active']:
    d = datetime.strptime(str(i),'%Y%m%d%H%M%S')
    time_string = d.strftime('%H:%M:%S')
    first_active_time.append(time_string)
    

In [ ]:
df['first_active_date']=first_active_date
df['first_active_time']=first_active_time
df=df.drop(['timestamp_first_active'], axis=1)

In [ ]:
fad = np.vstack(df['first_active_date'].astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
df['fad_year'] = fad[:, 0]
df['fad_month'] = fad[:, 1]
df['fad_day'] = fad[:, 2]
df = df.drop(['first_active_date'], axis=1)

In [ ]:
fad = np.vstack(df['first_active_time'].astype(str).apply(lambda x: list(map(int, x.split(':')))).values)
df['fad_hour'] = fad[:, 0]
df = df.drop(['first_active_time'], axis=1)

In [ ]:
dft=df[df['Train']==1]

In [ ]:
year_counts = dft.groupby('fad_year').count()['date_account_created'].to_frame().reset_index()
ax = sns.barplot(x = 'fad_year' , y='date_account_created', data = year_counts)

2013'te uygulamayı keşfeden ve giriş yapan daha fazla gibi görünüyor.

In [ ]:
month_counts = dft.groupby('fad_month').count()['date_account_created'].to_frame().reset_index()
bx = sns.barplot(x = 'fad_month' , y='date_account_created', data = month_counts)

Genellikle ilk 6 ayda daha fazla kullanılmış

In [ ]:
day_counts = dft.groupby('fad_day').count()['date_account_created'].to_frame().reset_index()
cx = sns.barplot(x = 'fad_day' , y='date_account_created', data = day_counts)

Ayın son gününde daha az bir kullanım görünüyor. Bunun nedeni olarak aklıma ilk her ayın 31 günü olmaması geldi.

In [ ]:
plt.figure(figsize=(15,7))
dft.groupby(['fad_year','fad_month']).count()['date_account_created'].plot.bar()

airbnb'nin popülerleşmesi ile kullanımda artan bir trend var.

In [ ]:
hour_nums = dft.groupby(['fad_hour']).count()['date_account_created'].to_frame().reset_index()
dx = sns.barplot(x = 'fad_hour' , y="date_account_created", data = hour_nums, color=sns.xkcd_rgb["purple"])

Saat olarak öğlen uygulamada daha az aktif kişi görünüyor. Bunun nedeni insanların öğle aralarını kendilerine ayırmış (yemek yemek) olmaları olabilir.

train.dropna(subset =['date_first_booking'],inplace=True)
dfb = np.vstack(train['date_first_booking'].astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
train['dfb_year'] = dfb[:, 0]
train['dfb_month'] = dfb[:, 1]
train['dfb_day'] = dfb[:, 2]
train = train.drop(['date_first_booking'], axis=1)

In [ ]:
dest_nums =  dft.groupby('country_destination').count()['gender'].sort_values(ascending = False).to_frame().reset_index()
hx = sns.barplot(x = 'country_destination' , y="gender", data = dest_nums)

In [ ]:
dac = np.vstack(df['date_account_created'].astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
df['dac_year'] = dac[:, 0]
df['dac_month'] = dac[:, 1]
df['dac_day'] = dac[:, 2]
df = df.drop(['date_account_created'], axis=1)

In [ ]:
dft=df[df['Train']==1]

In [ ]:
dac_year = dft.groupby(['dac_year']).count()['country_destination'].to_frame().reset_index()
dx = sns.barplot(x = 'dac_year' , y="country_destination", data = dac_year, color=sns.xkcd_rgb["purple"])

Gender datasında bilmediğimiz çok data var -unknown- olarak. Cinsiyeti önemli bir feature olarak düşündüğüm için bunu nasıl dolduracağıma baktım.

In [ ]:
fig=plt.figure(figsize=(5,5))
sns.countplot(x='gender', data=dft)

In [ ]:
fig=plt.figure(figsize=(10,10))
sns.countplot(x='country_destination', data=dft, hue ='gender')

In [ ]:
Herhangi bir özelliğin seçenekleri ile cinsiyetin belirgin bir ilişkisini göremedim. 
Bu nedenle modelimde -unkonwn- olarak bıraktım. Daha önce knnimputer da denedim ama sonuçlarımı çok etkilemedi.